# ライブラリのインポート

In [653]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_splitx

# データの準備

## データの読み込み

datasetの使い方は[こちら](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)

In [654]:
iris = load_iris()

In [655]:
inputs = iris.data
targets = iris.target
feature_names = iris.feature_names
target_names  = iris.target_names

In [656]:
print("説明変数：{}".format(feature_names))
print("目的変数：{}".format(target_names))

説明変数：['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
目的変数：['setosa' 'versicolor' 'virginica']


## trainとtestに分ける

train_test_splitの使い方は[こちら](https://docs.pyq.jp/python/machine_learning/tips/train_test_split.html)

In [657]:
# データセットを訓練データとテストデータに分ける
X_train, X_test, y_train, y_test = \
train_test_split(iris.data, iris.target, test_size=0.3)

# CARTの実装

## 特徴量と条件分岐の閾値の決定

### Gini係数の算出

決定木は『不純度』を用いることで分岐条件や特徴量を選択する．
不純度とは，ノード分岐の条件や選択した特徴量(説明変数)によって，ノードに分岐したサンプルのクラスがどれだけ散らばるかを表す指標の1つである．
分岐した際に，クラスがばらけることなく，あるノードに1種類のクラスだけが分類された場合(きれいに分割できた場合)，**そのノードの不純度は0となる**．
不純度を算出する方法は『交差エントロピー』と『Gini係数』があり，CARTの場合は『Gini係数』を用いることで不純度の計算を行う．
Gini係数は，ノード$t$におけるクラス$C_{i}$の割合を$P^{2}(C_{i}|t)$とすると，
$$
1 - \sum_{i=1}^{K}P^{2}(C_{i}|t),
$$
で算出することができる．
Gini係数は，ある条件に対して"Yes"と"No"の，2つ場合に対して求める必要がある(後にGini不純度を計算する必要があるため)．
詳しくは[こちら](https://hktech.hatenablog.com/entry/2018/10/05/004235)．

### Gini不純度の算出

Gini不純度とは，分岐条件毎に算出していた『Gini係数』を用いることで，その分岐条件自体を評価するための指標である．Gini不純度を用いることで，その特徴量における最も分類精度が高い分岐条件を求めることができるので，全特徴量のGini不純度を比較することで，分岐条件だけでなく分類精度が最も高くなる特徴量を選出することも可能となる．ある特徴量における分岐条件のGini不純度は，ノードの集合を$\{\mathrm{yes},\mathrm{no}\}$，1つ前のノードのデータ数を$A$，各ノードのデータ数を$a$とすると，
$$
\sum_{n \in \{\mathrm{yes},\mathrm{no}\}}\frac{a_n}{A} \times \mathrm{Gini}(n),
$$
で算出することができる．ここで$\mathrm{Gini}(n)$とは，yes or no のノードのGini係数である．

### 利得(gain)の算出

利得(gain)とは，分岐前のGini不純度から分岐後のGini不純度を引いた値である．これは分岐後のGini不純度が大きく下がっていれば，それだけその分岐が有効であったことがわかるので，利得が大きければ大きいほどその分岐が重要だということが示される．また各特徴量の分岐の利得を比較すれば，最適な特徴量空間を選出することも可能となる．本来であれば利得を用いて特徴量と閾値を算出すべきだが，今回は利得ではなく各特徴量のGini不純度を比較することでそれらを選出する．

In [658]:
class Cart(object):
    """
    これは，run()を実行すると"Gini不純度が最も低かった特徴量のindex"と"その時の閾値"を渡してくれるクラスである．
    今回はirisのデータセットを用いているが，入力データやターゲットを変えていただければ
    別のsklearnのデータセットなどでも適用することが可能である．
    """
    def __init__(self):
        self.inputs = None
        self.targets = None
        self.feature_names = None
        self.target_names = None

        self.gini = None
        self.node = None
        self.yes_num = None
        self.no_num = None
        
        self.impurity = None
        
        self.min_imp = None
        self.min_gini = None
        self.threshold = None
        
    
    def calc_gini(self,input_1d):
        """
        ある特徴量に対する全データのGini係数を算出する(2.1参照)．
        
        Parameters
        -------------------------------------------
        input_1d : ndarray
            ある特徴量における現在いるノードの全データである
        -------------------------------------------
        """
        self.gini = np.zeros((len(input_1d),2))
        
        for i, data in enumerate(input_1d):
            branch = input_1d > data
            self.node = np.zeros(2)
            self.yes_num = np.sum(branch)
            self.no_num  = np.sum(np.logical_not(branch))            
            if self.yes_num == 0:
                self.yes_num = 1e-8
            if self.no_num == 0:
                self.no_num = 1e-8
                     
            for c in range(len(self.target_names)):
                self.node[0] += np.square(np.sum(self.targets[branch]==c) / self.yes_num)
                self.node[1] += np.square(np.sum(self.targets[np.logical_not(branch)]==c) / self.no_num)
            for j in range(2):
                self.gini[i,j] = 1 - self.node[j]
    
    
    def calc_impurity(self,input_1d):
        """
        ある特徴量に対する全データのGini係数から，
        分岐の良し悪しを評価するGini不純度を算出し，最小のGini不純度を獲得する(2.2参照)．
        最小のGini不純度を獲得することで，最適な分岐条件と特徴量を決定することが可能となる(2.3参照)．
        
        Parameters
        -------------------------------------------
        input_1d : ndarray
            ある特徴量における現在いるノードの全データである
        -------------------------------------------
        """
        self.calc_gini(input_1d)
        
        self.impurity = np.zeros(len(self.gini))
        self.impurity = self.gini[:,0] * (self.yes_num / (self.yes_num + self.no_num)) + \
                        self.gini[:,1] * (self.no_num  / (self.yes_num + self.no_num))
    
    
    def best_impurity(self):
        """
        ある特徴量での，Gini不純度が最も小さい時の値とそのindexを返す．
        
        Returns
        -------------------------------------------
        np.min(self.impurity) : int
            ある特徴量における入力された全データの中でGini不純度が最も小さい時の値．
        np.argmin(self.impurity) : int
            ある特徴量における入力された全データの中でGini不純度が最も小さい時のindex．
        -------------------------------------------
        """
        return np.min(self.impurity), np.argmin(self.impurity)
    
    
    def best_feature(self):
        """
        全特徴量のGini不純度を計算し比較することで，
        クラスが最も綺麗に分かれる特徴量と分岐の閾値を獲得する(2.3参照)．
        
        Returns
        -------------------------------------------
        self.min_gini[np.argmin(self.min_imp[:,0])] : int
            全特徴量の中でGini不純度が最も小さい特徴量のGini係数
        np.argmin(self.min_imp[:,0]) : int
            全特徴量の中でGini不純度が最も小さい特徴量のindex．
        self.threshold : float
            全特徴量の中でGini不純度が最も小さい特徴量の閾値．
        -------------------------------------------
        """
        self.min_imp = np.zeros((len(self.inputs[0]),2))
        self.threshold = 0
        
        for i in range(len(self.inputs[0])):
            self.calc_impurity(self.inputs[:,i])
            self.min_imp[i,0], self.min_imp[i,1] = self.best_impurity()
        
        self.threshold = self.inputs[self.min_imp[np.argmin(self.min_imp[:,0]),1].astype('int64'), \
                                     np.argmin(self.min_imp[:,0])]
        
        return np.min(self.min_imp[:,0]), np.argmin(self.min_imp[:,0]), self.threshold
        
    
    def search_best_split(self,inputs,targets,feature_names,target_names):
        """
        Gini係数とGini不純度の計算を行う．
        最終的に，クラスが最も綺麗に分かれる特徴量と分岐の閾値を獲得する．

        Parameters
        -------------------------------------------
        inputs : ndarray
            全特徴量におけるデータの総数．
            irisでは，1回目は[105,4]のリストが入るが，
            2回目以降はそのノードにおけるデータの総数となる．
        targets : ndarray
            全特徴量における教師ラベル．
            irisでは，1回目は105のサイズだが，
            2回目以降はそのノードにおけるデータの総数がサイズとなる．
        feature_names : list
            特徴量の名前.
            irisでは[4,0]のリストを代入する
        target_names : ndarray
            クラスの名前．
            irisでは[3,0]のリストを代入する
        -------------------------------------------
        
        Returns
        -------------------------------------------
        self.selc_feature() : float, int, float
            全特徴量の中でGini不純度が最も小さい特徴量のGini不純度
            全特徴量の中で最もGini不純度が小さい特徴量のindex
            その時の特徴量の分岐の閾値．
        -------------------------------------------
        """
        self.inputs = inputs
        self.targets = targets
        self.feature_names = feature_names
        self.target_names = target_names
                
        return self.best_feature()
        

## 決定木の構築

### 決定木のノードの構築

In [659]:
class DecisionTreeNode(object):
    def __init__(self,inputs,targets,feature_names,target_names,algorithm,max_depth):
        self.inputs = inputs
        self.targets = targets
        self.feature_names = feature_names
        self.target_names = target_names
        self.algorithm = algorithm
        
        self.f_idx = None
        self.impurity = None
        self.threshold = None
        
        self.label = None
        self.depth = None
        self.max_depth = max_depth
        
        self.left = None
        self.right = None
    
    def split(self,depth):
        self.depth = depth
        if len(self.targets) != 0:
            self.label = np.argmax(np.bincount(self.targets))
        if len(self.inputs) == 0 :
            return
        
        self.impurity, self.f_idx, self.threshold = \
        self.algorithm.search_best_split(self.inputs,self.targets,self.feature_names,self.target_names)
        
        print("深さ:{}, 最適な特徴量:{}, 分岐条件の閾値:{}, このノードのクラス:{}".format( \
               self.depth, self.feature_names[self.f_idx], self.threshold, self.target_names[self.label]))
        
        if self.depth==self.max_depth or self.impurity == 0.0 :
            return
        
        left_idx = self.inputs[:,self.f_idx] <= self.threshold
        right_idx = self.inputs[:,self.f_idx] > self.threshold
        
        self.left = DecisionTreeNode(self.inputs[left_idx],self.targets[left_idx], \
                                     self.feature_names,self.target_names,self.algorithm,self.max_depth)
        self.left.split(self.depth+1)
        
        self.right = DecisionTreeNode(self.inputs[right_idx],self.targets[right_idx], \
                                      self.feature_names,self.target_names,self.algorithm,self.max_depth)
        self.right.split(self.depth+1)
        
    def predict(self,test_data):
        if self.depth == self.max_depth or self.impurity == 0.0 or len(self.inputs) == 0:
            return self.label
        
        if test_data[self.f_idx] <= self.threshold :
            return self.left.predict(test_data)
        elif test_data[self.f_idx] > self.threshold :
            return self.right.predict(test_data)

### 決定木の学習とテスト

In [660]:
class DecisionTreeClassifier(object):
    def __init__(self,inputs,targets,feature_names,target_names,algorithm,max_depth=3):
        self.inputs = inputs
        self.targets = targets
        self.feature_names = feature_names
        self.target_names = target_names
        self.algorithm = algorithm
        self.max_depth = max_depth
        self.tree = None
        
    def learning(self):
        self.tree = DecisionTreeNode(self.inputs,self.targets, \
                                     self.feature_names,self.target_names,self.algorithm,self.max_depth)
        self.tree.split(0)
        
    def predict(self,test_data):
        pred = []
        for data in test_data:
            pred.append(self.tree.predict(data))
        return np.array(pred)

## 実行

In [661]:
clf = DecisionTreeClassifier(X_train,y_train,feature_names,target_names,Cart(),4)
clf.learning()
y_pred = clf.predict(X_test)

深さ:0, 最適な特徴量:sepal length (cm), 分岐条件の閾値:4.8, このノードのクラス:virginica
深さ:1, 最適な特徴量:sepal length (cm), 分岐条件の閾値:4.4, このノードのクラス:setosa
深さ:1, 最適な特徴量:sepal width (cm), 分岐条件の閾値:2.0, このノードのクラス:virginica
深さ:2, 最適な特徴量:sepal length (cm), 分岐条件の閾値:5.0, このノードのクラス:versicolor
深さ:3, 最適な特徴量:sepal length (cm), 分岐条件の閾値:5.0, このノードのクラス:versicolor
深さ:4, 最適な特徴量:sepal length (cm), 分岐条件の閾値:5.0, このノードのクラス:versicolor
深さ:2, 最適な特徴量:petal width (cm), 分岐条件の閾値:0.6, このノードのクラス:virginica
深さ:3, 最適な特徴量:sepal length (cm), 分岐条件の閾値:5.7, このノードのクラス:setosa
深さ:3, 最適な特徴量:petal width (cm), 分岐条件の閾値:1.7, このノードのクラス:virginica
深さ:4, 最適な特徴量:petal width (cm), 分岐条件の閾値:1.0, このノードのクラス:versicolor
深さ:4, 最適な特徴量:sepal length (cm), 分岐条件の閾値:5.6, このノードのクラス:virginica


# 結果

In [662]:
score = sum(y_pred == y_test) / len(y_test)
print("classification accuracy : {}".format(score))

classification accuracy : 0.9777777777777777


# 参考文献

- [sklearn.datasets.load_iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html)
- [train_test_split関数でデータ分割](https://docs.pyq.jp/python/machine_learning/tips/train_test_split.html)
- [Pythonで決定木分類器をフルスクラッチで実装してみた](https://hktech.hatenablog.com/entry/2018/10/05/004235)
- [[入門]初心者の初心者による初心者のための決定木分析](https://qiita.com/3000manJPY/items/ef7495960f472ec14377)